In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

<a href="https://www.youtube.com/watch?v=_ediOdFUr10"> Intro to Qchem pennylane </a>

<a href="https://www.youtube.com/watch?v=TiQ7T1h8VAQ"> Optimizing a quantum circuit pennylane </a>

# Step 1: Find Molecular Hamiltonian

In [3]:
symbols = ["H","H","H"]
coordinates = np.array([[0.0102,0.0442,0.0],
                        [0.9867,1.6303,0.0],[1.8720,-0.0085,0.0]]) # nuclei location in x,y,z in H3 atom
hamiltonian,qubits = qchem.molecular_hamiltonian(symbols,coordinates,charge=1) # cz in vqe it is better to work with 2e than 3e. So takin H3+
print(qubits) # see, we need 6 electrons to find h3 ground state

6


In [5]:
# now we need to define approx ground state known as Hartree Fock State
hf = qchem.hf_state(electrons=2,orbitals=6) # orbitals same as num of qubits needed
print(hf)

[1 1 0 0 0 0]


In [7]:
num_wires = qubits
dev=qml.device('default.qubit',wires=num_wires)

@qml.qnode(dev)
def exp_energy(state):
    qml.BasisState(np.array(state),wires=range(num_wires))
    return qml.expval(hamiltonian)
exp_energy(hf)

tensor(-1.24655016, requires_grad=True)

# Step 2: Prepare Trial Ground State

In [8]:
def ansatz(params):
    qml.BasisState(hf,wires=range(num_wires))
    qml.DoubleExcitation(params[0],wires=[0,1,2,3])
    qml.DoubleExcitation(params[1],wires=[0,1,4,5])
    

# Step 3: Minimize <H> ( expectation value of hamiltonian)

<b><u>Ritz Variational Principle:</u></b> If i find the state that minimizes the expectation value of hamiltonian then the state is ground state

In [9]:
@qml.qnode(dev)
def cost_function(params):
    ansatz(params)
    return qml.expval(hamiltonian)
cost_function([0.1,0.1]) # the output will be lower than hf expval

tensor(-1.26796721, requires_grad=True)

In [11]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
theta = np.array([0.0,0.0],requires_grad=True) # require_grad is used to specify if the tensor should track all opr on it, which allows 
# for the computational gradients
energy=[cost_function(theta)]
angle=[theta]
max_iterations=20
for n in range(max_iterations):
    theta,prev_energy =opt.step_and_cost(cost_function,theta)
    energy.append(cost_function(theta))
    angle.append(theta)
    if n%2==0:
        print(f'Step={n} , Energy = {energy[-1]:.8f} Ha')



Step=0 , Energy = -1.26070025 Ha
Step=2 , Energy = -1.27115671 Ha
Step=4 , Energy = -1.27365804 Ha
Step=6 , Energy = -1.27425241 Ha
Step=8 , Energy = -1.27439362 Ha
Step=10 , Energy = -1.27442718 Ha
Step=12 , Energy = -1.27443517 Ha
Step=14 , Energy = -1.27443707 Ha
Step=16 , Energy = -1.27443752 Ha
Step=18 , Energy = -1.27443763 Ha


In [12]:
print("\n" f"Final ground energy: {energy[-1]:.8f} Ha")



Final ground energy: -1.27443764 Ha


In [13]:
@qml.qnode(dev)
def ground_state(params):
    ansatz(params)
    return qml.state()
ground_state(theta)

tensor([ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09585862+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09586987+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.99076743+0.j,  0.        +0.j,  0.        +0.